# BERT Word Embeddings


In [7]:
!pip install torch
!pip install transformers
!pip install stanza
!pip install rowordnet

In [8]:
from transformers import BertTokenizer, BertModel
import pandas as pd
import numpy as np
import nltk
import torch
import stanza
import pickle
import math
from scipy.spatial.distance import cosine
import rowordnet as rwn
import csv

In [9]:
from google.colab import data_table
data_table.enable_dataframe_formatter()

In [10]:
# Loading the pre-trained BERT model
###################################
# Embeddings will be derived from
# the outputs of this model
model = BertModel.from_pretrained('dumitrescustefan/bert-base-romanian-uncased-v1',
                                  output_hidden_states = True,
                                  )

# Setting up the tokenizer
###################################
# This is the same tokenizer that
# was used in the model to generate 
# embeddings to ensure consistency
tokenizer = BertTokenizer.from_pretrained('dumitrescustefan/bert-base-romanian-uncased-v1')

Some weights of the model checkpoint at dumitrescustefan/bert-base-romanian-uncased-v1 were not used when initializing BertModel: ['cls.predictions.transform.dense.bias', 'cls.predictions.transform.dense.weight', 'cls.seq_relationship.bias', 'cls.predictions.decoder.bias', 'cls.predictions.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.seq_relationship.weight', 'cls.predictions.decoder.weight']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [11]:
# Text corpus
##############

!gdown --id 1IV_nodlm-dw-EWl1DtngkATgAldEdAGO

with open("dataset.pickle", "rb") as pickleFile:
    db = pickle.load(pickleFile)

Downloading...
From: https://drive.google.com/uc?id=1IV_nodlm-dw-EWl1DtngkATgAldEdAGO
To: /content/dataset.pickle
100% 93.3M/93.3M [00:00<00:00, 181MB/s]


In [12]:
stanza.download('ro')
stz = stanza.Pipeline('ro')
wn = rwn.RoWordNet()

2022-01-25 18:32:38 INFO: Downloading default packages for language: ro (Romanian)...


2022-01-25 18:32:47 INFO: Finished downloading models and saved to /root/stanza_resources.
2022-01-25 18:32:47 INFO: Loading these models for language: ro (Romanian):
| Processor | Package |
-----------------------
| tokenize  | rrt     |
| pos       | rrt     |
| lemma     | rrt     |
| depparse  | rrt     |

2022-01-25 18:32:47 INFO: Use device: gpu
2022-01-25 18:32:47 INFO: Loading: tokenize
2022-01-25 18:32:59 INFO: Loading: pos
2022-01-25 18:33:00 INFO: Loading: lemma
2022-01-25 18:33:00 INFO: Loading: depparse
2022-01-25 18:33:01 INFO: Done loading processors!


In [13]:
def parse_sentence(s):
  s = s.replace("ţ", "ț").replace("ş", "ș").replace("Ţ", "Ț").replace("Ş", "Ș")
  result = []
  sres = ""
  doc = stz(s)
  for _, sentence in enumerate(doc.sentences):
      for token in sentence.tokens:
          for word in token.words:
              result.append(word.lemma)
  for w in result:
    sres += w + " "
  return sres

In [14]:
def bert_text_preparation(text, tokenizer):
    """Preparing the input for BERT
    
    Takes a string argument and performs
    pre-processing like adding special tokens,
    tokenization, tokens to ids, and tokens to
    segment ids. All tokens are mapped to seg-
    ment id = 1.
    
    Args:
        text (str): Text to be converted
        tokenizer (obj): Tokenizer object
            to convert text into BERT-re-
            adable tokens and ids
        
    Returns:
        list: List of BERT-readable tokens
        obj: Torch tensor with token ids
        obj: Torch tensor segment ids
    
    
    """

    marked_text = "[CLS] " + parse_sentence(text) + " [SEP]"
    tokenized_text = tokenizer.tokenize(marked_text)
    indexed_tokens = tokenizer.convert_tokens_to_ids(tokenized_text)
    segments_ids = [1]*len(indexed_tokens)

    # Convert inputs to PyTorch tensors
    tokens_tensor = torch.tensor([indexed_tokens])
    segments_tensors = torch.tensor([segments_ids])

    return tokenized_text, tokens_tensor, segments_tensors
    
def get_bert_embeddings(tokens_tensor, segments_tensors, model):
    """Get embeddings from an embedding model
    
    Args:
        tokens_tensor (obj): Torch tensor size [n_tokens]
            with token ids for each token in text
        segments_tensors (obj): Torch tensor size [n_tokens]
            with segment ids for each token in text
        model (obj): Embedding model to generate embeddings
            from token and segment ids
    
    Returns:
        list: List of list of floats of size
            [n_tokens, n_embedding_dimensions]
            containing embeddings for each token
    
    """
    
    # Gradient calculation id disabled
    # Model is in inference mode
    with torch.no_grad():
        outputs = model(tokens_tensor, segments_tensors)
        # Removing the first hidden state
        # The first state is the input state
        hidden_states = outputs[2][1:]

    # Getting embeddings from the final BERT layer
    token_embeddings = hidden_states[-1]
    # Collapsing the tensor into 1-dimension
    token_embeddings = torch.squeeze(token_embeddings, dim=0)
    # Converting torchtensors to lists
    list_token_embeddings = [token_embed.tolist() for token_embed in token_embeddings]

    return list_token_embeddings

In [15]:
def strip_diacritics(word):
  word = word.lower()
  word = word.replace("ă", "a").replace("â", "a").replace("ș", "s").replace("ț", "t").replace("î", "i")

  return word

In [16]:
def get_embedding_of_word(word, text):
  tokenized_text, tokens_tensor, segments_tensors = bert_text_preparation(text, tokenizer)
  tokenized_text_word, tokens_tensor_word, segments_tensors_word = bert_text_preparation(word, tokenizer)
  # print(tokenized_text)
  # print(tokenized_text_word)
  list_token_embeddings = get_bert_embeddings(tokens_tensor, segments_tensors, model)
  word_index = tokenized_text.index(tokenized_text_word[1])
  word_embedding = list_token_embeddings[word_index]
  return word_embedding

In [17]:
def get_word_statistics(word, sentence_by_word, logging=False, output_csv=None):
  correct_finds = 0
  finds = 0
  embeddings_test = []
  embeddings_train = []

  if output_csv is not None:
    out = open(output_csv + ".txt", "at", encoding="utf-8")
    csv_out = open(output_csv + ".csv",
                   "a", encoding="utf-8", newline='')
    writer = csv.writer(csv_out)

  for sid in sentence_by_word[word]:
    avg_train_embedding = [0.0] * 768
    if sid == "-1":
      continue
    nr_per_sid = len(sentence_by_word[word][sid])
    if nr_per_sid < 2:
      continue
    limit = math.ceil(0.2 * nr_per_sid)

    try:
      for i in range(limit):
        text = sentence_by_word[word][sid][i]
        word_embedding = get_embedding_of_word(word, text)

        embeddings_test.append({"correct_synset_id": sid, "embedding": word_embedding, "sentence": text})

      for i in range(limit, nr_per_sid):
        text = sentence_by_word[word][sid][i]
        word_embedding = get_embedding_of_word(word, text)
        for k in range(len(word_embedding)):
          avg_train_embedding[k] += word_embedding[k]

      for k in range(len(word_embedding)):
        avg_train_embedding[k] /= nr_per_sid - limit
      embeddings_train.append({"correct_synset_id": sid, "avg_embedding": avg_train_embedding})
    except:
      continue

  list_of_distances = []

  for test_text in embeddings_test:
    max_dist = 0
    best_sense = None
    for train_text in embeddings_train:
      cos_dist = 1 - cosine(test_text["embedding"], train_text["avg_embedding"])
      if cos_dist > max_dist:
        max_dist = cos_dist
        best_sense = train_text["correct_synset_id"]
      list_of_distances.append([test_text["sentence"], wn.synset(train_text["correct_synset_id"]).definition, cos_dist])
    if best_sense == test_text["correct_synset_id"]:
      correct_finds += 1
    finds += 1
    if output_csv:
      writer.writerow([word, test_text["sentence"], test_text["correct_synset_id"], best_sense, best_sense])
  

  if logging is True:
    print(word, correct_finds, finds)
    distances_df = pd.DataFrame(list_of_distances, columns=['sentence', 'synset_def', 'distance'])
    for test in embeddings_test:
      print("Marked as correct: " + wn.synset(test["correct_synset_id"]).definition)
      display(distances_df[distances_df.sentence == test["sentence"]])

  if output_csv is not None:
    out.write('Correct guesses for word ' + word + ": " +
                        str(correct_finds) + "/" + str(finds) + "\n")
    out.close

  return correct_finds, finds

In [18]:
def get_word_statistics_for_sentence(word, given_sentence, sentence_by_word, logging=False):
  correct_finds = 0
  finds = 0
  embeddings_test = []
  embeddings_train = []

  for sid in sentence_by_word[word]:
    avg_train_embedding = [0.0] * 768
    if sid == "-1":
      continue
    nr_per_sid = len(sentence_by_word[word][sid])
    if nr_per_sid < 2:
      continue
    limit = math.ceil(0.2 * nr_per_sid)

    try:
      for i in range(limit, nr_per_sid):
        text = sentence_by_word[word][sid][i]
        word_embedding = get_embedding_of_word(word, text)
        for k in range(len(word_embedding)):
          avg_train_embedding[k] += word_embedding[k]

      for k in range(len(word_embedding)):
        avg_train_embedding[k] /= nr_per_sid - limit
      embeddings_train.append({"correct_synset_id": sid, "avg_embedding": avg_train_embedding})
    except:
      continue

  try:
    word_embedding = get_embedding_of_word(word, given_sentence)

    embeddings_test.append({"correct_synset_id": "-1", "embedding": word_embedding, "sentence": given_sentence})
  except:
    print("Could not tokenize sentence")

  list_of_distances = []

  for test_text in embeddings_test:
    max_dist = 0
    best_sense = None
    for train_text in embeddings_train:
      cos_dist = 1 - cosine(test_text["embedding"], train_text["avg_embedding"])
      if cos_dist > max_dist:
        max_dist = cos_dist
        best_sense = train_text["correct_synset_id"]
      list_of_distances.append([test_text["sentence"], wn.synset(train_text["correct_synset_id"]).definition, cos_dist])
    if best_sense == test_text["correct_synset_id"]:
      correct_finds += 1
    finds += 1
  

  if logging is True:
    distances_df = pd.DataFrame(list_of_distances, columns=['sentence', 'synset_def', 'distance'])
    for test in embeddings_test:
      try:
        definition = wn.synset(test["correct_synset_id"]).definition
      except:
        definition = "No known correct definition"
      print("Marked as correct: " + definition)
      display(distances_df[distances_df.sentence == test["sentence"]])

In [19]:
target_word_embeddings = []

sentence_by_word = dict()
for word in db:
  sentence_by_sid = dict()

  for sentence_id in range(len(db[word])):
    correct_sense = db[word][sentence_id]["correct_synset_id"]
    if correct_sense not in sentence_by_sid:
      sentence_by_sid[correct_sense] = []
    sentence_by_sid[correct_sense].append(db[word][sentence_id]["sentence"])
    
  sentence_by_word[word] = sentence_by_sid

In [20]:
print(get_word_statistics("reînviere", sentence_by_word, logging=True))

reînviere 2 2
Marked as correct: revenire din inactivitate


,sentence,synset_def,distance
0,Când muzica gregoriană a experimentat o reînvi...,revenire din inactivitate,0.850573
1,Când muzica gregoriană a experimentat o reînvi...,acțiunea de a reda funcțiile vitale ale organi...,0.762956


Marked as correct: acțiunea de a reda funcțiile vitale ale organismului, a repune în funcțiune inima sau respirația oprite, a readuce în simțiri, a face să își revină


,sentence,synset_def,distance
2,"Dacă distrugeți nodul de reînviere, Cylonii nu...",revenire din inactivitate,0.797910
3,"Dacă distrugeți nodul de reînviere, Cylonii nu...",acțiunea de a reda funcțiile vitale ale organi...,0.871887


(2, 2)


In [21]:
print(get_word_statistics_for_sentence("rol", "Tom Holland joacă rolul principal în Spiderman.", sentence_by_word, logging=True))

Marked as correct: No known correct definition


,sentence,synset_def,distance
0,Tom Holland joacă rolul principal în Spiderman.,"Atribuție, sarcină care îi revine cuiva în cad...",0.782320
1,Tom Holland joacă rolul principal în Spiderman.,Partea de muncă atribuită unei persoane sau un...,0.773388
2,Tom Holland joacă rolul principal în Spiderman.,Fiecare dintre persoane care figurează într-o ...,0.859338
3,Tom Holland joacă rolul principal în Spiderman.,Partitură scenică ce revine unui actor într-o ...,0.839968
4,Tom Holland joacă rolul principal în Spiderman.,semnificație de ordin semantic sau lingvistic ...,0.747932


None


In [22]:
print(get_word_statistics_for_sentence("rol", "BERT joacă un rol foarte important în WSD.", sentence_by_word, logging=True))

Marked as correct: No known correct definition


,sentence,synset_def,distance
0,BERT joacă un rol foarte important în WSD.,"Atribuție, sarcină care îi revine cuiva în cad...",0.835584
1,BERT joacă un rol foarte important în WSD.,Partea de muncă atribuită unei persoane sau un...,0.820374
2,BERT joacă un rol foarte important în WSD.,Fiecare dintre persoane care figurează într-o ...,0.685536
3,BERT joacă un rol foarte important în WSD.,Partitură scenică ce revine unui actor într-o ...,0.685834
4,BERT joacă un rol foarte important în WSD.,semnificație de ordin semantic sau lingvistic ...,0.658328


None


In [23]:
correct_finds_total = 0
finds_total = 0

for word in db:
  nr_s = len(db[word][0]['synsets'].split())
  if len(db[word]) > 100 and nr_s <= 5:
    correct_finds, finds = get_word_statistics(word, sentence_by_word)
    
    correct_finds_total += correct_finds
    finds_total += finds
    print(word, correct_finds, finds)

print(correct_finds_total, finds_total)


prietenie 5 10
top 1 1
punere 2 3
grupă 2 2
sită 2 2
acumulare 8 13
cabană 16 20
latin 1 4
deget 6 6
impozit 34 35
cinste 7 8
furnizare 38 38
fibră 5 9
bilă 7 8
culme 5 8
alimentare 0 0
invazie 37 52
email 2 3
chei 6 10
C 4 5
autostradă 23 23
lord 0 0
luare 6 9
prânz 7 10
rugăciune 5 9
întrunire 6 7
ocol 5 9
240 304


In [24]:
print(get_word_statistics_for_sentence("cafeniu", "Ursul cafeniu e fioros.", sentence_by_word, logging=True))

Marked as correct: No known correct definition


,sentence,synset_def,distance
0,Ursul cafeniu e fioros.,nuanță închisă de maro,0.825754


None


In [27]:
def create_bert_files(words, filename):
  out = open(filename + ".txt", "wt", encoding="utf-8")
  csv_out = open(filename + ".csv",
                  "w", encoding="utf-8", newline='')
  header = ['word', 'sentence_id', 'correct_synset_id',
            'predicted_synsets', 'chosen_synset']
  writer = csv.writer(csv_out)
  writer.writerow(header)
  out.close()
  csv_out.close()

  for word in words:
    get_word_statistics(word, sentence_by_word, output_csv=filename)

In [29]:
words = []

for i in range(2, 36):
  no_words = 0
  for word in db:
    no_senses = len(db[word][0]['synsets'].split()) - 1
    if no_senses == i:
      no_words += 1
      words.append(word)
    if no_words == 5:
      break

print(words)

# words = ["bancă"]

create_bert_files(words, "bert")

['secol', 'județ', 'comună', 'uniune', 'muncă', 'persoană', 'locuitor', 'biserică', 'nord', 'teritoriu', 'armată', 'film', 'stat', 'majoritate', 'activitate', 'rol', 'oraș', 'mod', 'echipă', 'război', 'dată', 'perioadă', 'companie', 'om', 'an', 'fapt', 'problemă', 'lună', 'membru', 'plan', 'măsură', 'interior', 'prezent', 'urmare', 'familie', 'nevoie', 'lume', 'regiune', 'apă', 'piață', 'sistem', 'limbă', 'ban', 'nivel', 'grup', 'vedere', 'caz', 'zonă', 'dezvoltare', 'ajutor', 'nume', 'timp', 'casă', 'număr', 'viață', 'valoare', 'conducere', 'drum', 'schimbare', 'apărare', 'lucru', 'sfârșit', 'urmă', 'fel', 'program', 'zi', 'joc', 'cadru', 'formă', 'forță', 'gol', 'bătaie', 'fals', 'schimb', 'acord', 'cădere', 'aripă', 'atac', 'calm', 'poziție', 'fin', 'serviciu', 'mediu', 'masă', 'putere', 'adânc', 'prost', 'semn', 'loc', 'mare', 'față', 'cap', 'liber', 'linie', 'bază', 'rău', 'parte', 'bun', 'legătură', 'drept']


In [30]:
print(get_word_statistics("bancă", sentence_by_word, logging=True))

bancă 9 13
Marked as correct: Întreprindere financiară care efectuează operații de plată și de credit (și organizează circulația bănească).


,sentence,synset_def,distance
0,Nicolae Iorga nr.11 și-a început activitatea o...,Întreprindere financiară care efectuează opera...,0.814170
1,Nicolae Iorga nr.11 și-a început activitatea o...,Clădire unde își desfășoară activitatea o banc...,0.847123
2,Nicolae Iorga nr.11 și-a început activitatea o...,Scaun lung pentru două sau mai multe persoane.,0.638145


Marked as correct: Întreprindere financiară care efectuează operații de plată și de credit (și organizează circulația bănească).


,sentence,synset_def,distance
3,Banca a fost prezentă în România din noiembrie...,Întreprindere financiară care efectuează opera...,0.876948
4,Banca a fost prezentă în România din noiembrie...,Clădire unde își desfășoară activitatea o banc...,0.881027
5,Banca a fost prezentă în România din noiembrie...,Scaun lung pentru două sau mai multe persoane.,0.670316


Marked as correct: Întreprindere financiară care efectuează operații de plată și de credit (și organizează circulația bănească).


...

Marked as correct: Scaun lung pentru două sau mai multe persoane.


,sentence,synset_def,distance
33,"Tu și tu, acolo și stați pe această bancă.",Întreprindere financiară care efectuează opera...,0.749967
34,"Tu și tu, acolo și stați pe această bancă.",Clădire unde își desfășoară activitatea o banc...,0.750986
35,"Tu și tu, acolo și stați pe această bancă.",Scaun lung pentru două sau mai multe persoane.,0.774911


Marked as correct: Scaun lung pentru două sau mai multe persoane.


,sentence,synset_def,distance
36,"În afară de activitatea de marinar, amiralul V...",Întreprindere financiară care efectuează opera...,0.467779
37,"În afară de activitatea de marinar, amiralul V...",Clădire unde își desfășoară activitatea o banc...,0.435313
38,"În afară de activitatea de marinar, amiralul V...",Scaun lung pentru două sau mai multe persoane.,0.647970


(9, 13)
